## Always run thic CELL below
- To attach ESML controlplane to your project
- To point at `template-data` for the pipelinbe to know the schema of data
- To init the ESMLPieplinefactory

In [2]:
import sys
sys.path.insert(0, "../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLProject
from baselayer_azure_ml_pipeline import ESMLPipelineFactory, esml_pipeline_types

p = ESMLProject()
p.inference_mode = False
p.active_model = 11 # 10=titanic , 11=Diabetes
p_factory = ESMLPipelineFactory(p)

scoring_date = '1000-01-01 10:35:01.243860' # 
p_factory.batch_pipeline_parameters[1].default_value = scoring_date # overrides ESMLProject.date_scoring_folder.
p_factory.describe()


Using lake_settings.json with ESML version 1.4 - Models array support including LABEL

 ---- Q: WHICH files are generated as templates, for you to EDIT? ---- 
A: These files & locations:
File to EDIT (step: IN_2_SILVER_1): ../../../2_A_aml_pipeline/4_inference/batch/M11/in2silver_ds01_diabetes.py
File to EDIT (step: IN_2_SILVER_2): ../../../2_A_aml_pipeline/4_inference/batch/M11/in2silver_ds02_other.py
File to EDIT (step: SILVER_MERGED_2_GOLD): ../../../2_A_aml_pipeline/4_inference/batch/M11/silver_merged_2_gold.py
File to EDIT (step: SCORING_GOLD): ../../../2_A_aml_pipeline/4_inference/batch/M11/scoring_gold.py
File to EDIT (step: TRAIN_SPLIT_AND_REGISTER): ../../../2_A_aml_pipeline/4_inference/batch/M11/train_split_and_register.py
File to EDIT (step: TRAIN_MANUAL): ../../../2_A_aml_pipeline/4_inference/batch/M11/train_manual.py
File to EDIT (step: TRAIN_AUTOML): ../../../2_A_aml_pipeline/4_inference/batch/M11/train_post_automl_step.py
File to EDIT a lot (reference in step-scripts Cus

# IN_2_GOLD_TRAIN_AUTOML
- 1) Generate code files
- 2) Build pipeline, ESML autoguild this, and will upload the snapshot folder together with the Azure ML pipeline.
- 3) Run the pipeline. Smoke testing, see that it works
- 4) IF it works, Publish the pipeline, or else, edit the code files or configuration, retry step 2 and 3.
- 5) Print the pipeline_id, that is essential to use from Azure Data factory 

In [2]:
# If you want to (not recommended) override default CURATED environment = "ESML-AzureML-144-AutoML_126"
#p_factory.use_curated_automl_environment = True
#p_factory.environment_name = "AzureML-AutoML" # Training[ "AzureML-AutoML", "AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu"]  Inference["AzureML-sklearn-0.24.1-ubuntu18.04-py37-cpu-inference","AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu"]

In [3]:
## Generate CODE - then edit it to get correct environments
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=True) # Do this once, then edit them manually. overwrite_if_exists=False is DEFAULT

Did NOT overwrite script-files with template-files such as 'scoring_gold.py', since overwrite_if_exists=False


In [3]:
p.get_all_envs() # Copy / Paste this info to the CUSTOMIZE section in generated file 'train_post_automl_step.py', set variable 'all_envs' to the below

{'dev': {'subscription_id': 'ca0a8c40-b06a-4e4e-8434-63c03a1dee34',
  'resourcegroup_id': 'MSFT-WEU-EAP_PROJECT02_AI-DEV-RG',
  'workspace_name': 'msft-weu-DEV-eap-proj02_ai-amls'},
 'test': {'subscription_id': 'ca0a8c40-b06a-4e4e-8434-63c03a1dee34',
  'resourcegroup_id': 'MSFT-WEU-EAP_PROJECT02_AI-TEST-RG',
  'workspace_name': 'msft-weu-TEST-eap-proj02_ai-amls'},
 'prod': {'subscription_id': 'ca0a8c40-b06a-4e4e-8434-63c03a1dee34',
  'resourcegroup_id': 'MSFT-WEU-EAP_PROJECT02_AI-PROD-RG',
  'workspace_name': 'msft-weu-PROD-eap-proj02_ai-amls'}}

In [ ]:
## BUILD
batch_pipeline = p_factory.create_batch_pipeline(esml_pipeline_types.IN_2_GOLD_TRAIN_AUTOML)
# ...which Trains a model on data via date_folder parameters, upload the generated python scripts., and your custom code and ESML runtime, to Azure embedded in the pipeline, using Dockerized image. 

In [5]:
## RUN
pipeline_run = p_factory.execute_pipeline(batch_pipeline)
pipeline_run.wait_for_completion(show_output=False)

Execute_pipeline (scoring): Inference_mode: 0
-Scoring data, default value 1000-01-01 10:35:01.243860
Created step IN 2 SILVER - ds01_diabetes [60525892][b22cc3ce-8498-40ab-ae47-862dec3c2f74], (This step will run and generate new outputs)Created step IN 2 SILVER - ds02_other [d90bbf2d][70708443-e3bf-42f1-b66b-212e6c301d06], (This step will run and generate new outputs)

Created step SILVER MERGED 2 GOLD [d54d24cd][fa22cbe4-11a3-4dd3-8443-17ea40322846], (This step will run and generate new outputs)
Created step SPLIT AND REGISTER datasets [301fcbda][05f43820-61ad-48d4-a5d0-eac5f5d1ac16], (This step will run and generate new outputs)
Created step AutoML TRAIN in  [dev] [eb3391dc][158b49d4-ba90-418d-b946-f6d3492332ea], (This step will run and generate new outputs)
Created step [dev]Calculate SCORING on TEST_SET, COMPARE & REGISTER model in [dev] & PROMOTE to [test] [8bc4f5f8][18d28ddc-b721-4fbb-9c65-c201e1a1f1cc], (This step will run and generate new outputs)
Submitted PipelineRun 6a84541

'Finished'

In [6]:
# PUBLISH
published_pipeline, endpoint = p_factory.publish_pipeline(batch_pipeline,"_1") # "_1" is optional    to create a NEW pipeline with 0 history, not ADD version to existing pipe & endpoint

pub_pipe.name 11_diabetes_model_reg_pipe_IN_2_GOLD_SCORING_EP_6
pub_pipe.id 5ecadc66-0895-4633-b7de-2fd3df5812ce
pub_pipe.name 11_diabetes_model_reg_pipe_IN_2_GOLD_EP
pub_pipe.id 8843bea2-7c7e-4738-9028-a16f89f7d6e7
pub_pipe.name 11_diabetes_model_reg_pipe_IN_2_GOLD_SCORING_EP_5
pub_pipe.id 5d828765-e496-4df6-9701-4581c31990e6
pub_pipe.name 11_diabetes_model_reg_batch_scoring_pipe_EP_4
pub_pipe.id e5ed3b30-89fd-4b70-a2a0-403451ea2228
pub_pipe.name 11_diabetes_model_reg_pipe_IN_2_GOLD
pub_pipe.id 5c1c0622-2f18-4380-865f-b5df380f2345
pub_pipe.name 10_titanic_model_clas_batch_scoring_pipe_EP_4
pub_pipe.id aa67ac38-da18-4b2d-973a-df87998aa1b6
pub_pipe.name 11_diabetes_model_reg_batch_scoring_pipe_EP_1
pub_pipe.id c21397ba-8e26-4582-b7eb-0cf9bee5cc0c


# Get info - to use  in Azure data factory: `published_pipeline.id` (if private Azure ML workspace)

In [7]:
print("2) Fetch scored data: Below needed for Azure Data factory PIPELINE activity (Pipeline OR Endpoint. Choose the latter") 
print ("- Endpoint ID")
print("Endpoint ID:  {}".format(endpoint.id))
print("Endpoint Name:  {}".format(endpoint.name))
print("Experiment name:  {}".format(p_factory.experiment_name))

print("In AZURE DATA FACTORY - This is the ID you need, if using PRIVATE LINK, private Azure ML workspace.")
print("-You need PIPELINE id, not pipeline ENDPOINT ID ( since cannot be chosen in Azure data factory if private Azure ML)")
published_pipeline.id

2) Fetch scored data: Below needed for Azure Data factory PIPELINE activity (Pipeline OR Endpoint. Choose the latter
- Endpoint ID
Endpoint ID:  681339bb-c47c-4af3-b408-df8155395f74
Endpoint Name:  11_diabetes_model_reg_pipe_IN_2_GOLD_TRAIN_EP_1
Experiment name:  11_diabetes_model_reg_pipe_IN_2_GOLD_TRAIN
In AZURE DATA FACTORY - This is the ID you need, if using PRIVATE LINK, private Azure ML workspace.
-You need PIPELINE id, not pipeline ENDPOINT ID ( since cannot be chosen in Azure data factory if private Azure ML)


'af7a8f47-7cda-47cc-afc7-c71cce7fecd6'